In [1]:
import pandas as pd

import numpy as np

import glob

import sympy as sym

In [3]:
import os

from importlib.machinery import SourceFileLoader

os.chdir('../')

main = SourceFileLoader("main", "src/main.py").load_module()

from main import *

In [4]:
eq = SourceFileLoader("eq", "src/numerical_equilibria_n_bit_vs_n_bit.py").load_module()

from eq import *

In [5]:
os.chdir("nbs/")

In [ ]:
files = glob.glob("../ada/two_bit_against_memory_two/*.csv")

In [ ]:
columns = (
    ["", "ID"]
    + [f"p{i+1}" for i in range(16)]
    + ["label", "Sp", "Sq", "condition A", "condition B", "c", "b"]
)

dfs = [pd.read_csv(file, index_col=0, names=columns) for file in files]

df = pd.concat(dfs)

In [6]:
files = glob.glob("../ada/memory_two_against_memory_two/*.csv")

columns = (
    ["", "ID"]
    + [f"p{i+1}" for i in range(16)]
    + ["label", "Sp", "Sq", "condition A", "condition B", "c", "b"]
)

dfs = [pd.read_csv(file, index_col=0, names=columns) for file in files]

df2 = pd.concat(dfs)

In [8]:
# df.to_csv("../ada/two_bit_against_memory_two_merged.csv")

**Equilibria and not**

In [7]:
def seperate_equilibria_and_non(df):
    per_player = df.groupby('ID')['condition A'].all()

    eq_players_ID = per_player[per_player == True].index

    noneq_players_ID = per_player[per_player == False].index
    
    return df[df['ID'].isin(eq_players_ID)], df[df['ID'].isin(noneq_players_ID)]

In [8]:
eqmemtwo, noneqmemtwo = seperate_equilibria_and_non(df2)

In [ ]:
df['ID'].nunique(), eqmemtwo['ID'].nunique(), noneqmemtwo['ID'].nunique()

In [11]:
df2['ID'].nunique(), eqmemtwo['ID'].nunique(), noneqmemtwo['ID'].nunique()

(922, 387, 535)

**The smallest set that explains equilibria**

In [9]:
import itertools
import collections

dimensions = 2

deterministic_strategies = list(
    itertools.product([0, 1], repeat=2 ** (2 * dimensions))
)

In [10]:
groups = noneqmemtwo[noneqmemtwo['condition A'] == False].groupby('ID')

sets, ids = [], []

for i, group in groups:
    sets.append(group['label'].values)
    ids.append(group['ID'].unique())

In [12]:
len(sets)

535

In [13]:
flat_list = [item for sublist in sets for item in sublist]

In [14]:
count = collections.Counter(flat_list)

In [17]:
max(count.values())

441

In [19]:
len((count.keys()))

39413

In [22]:
remaining = [s for s in sets if ('N0' not in s)]

In [23]:
flat_list = [item for sublist in remaining for item in sublist]

In [24]:
count = collections.Counter(flat_list)

In [25]:
len((count.keys()))

35766

In [20]:
search_parameters = [(51, 36),
 (50, 97),
 (49, 35),
 (48, 37),
 (47, 25),
 (46, 42),
 (45, 32),
 (44, 4),
 (43, 6),
 (42, 83),
 (41, 54),
 (40, 60),
 (39, 24),
 (38, 77),
 (37, 34),
 (36, 51),
 (35, 83),
 (34, 79),
 (33, 72),
 (32, 68),
 (31, 221),
 (30, 232),
 (29, 415),
 (28, 200),
 (27, 210),
 (26, 189),
 (25, 214),
 (24, 259),
 (23, 246),
 (22, 282)]

In [21]:
sum(k[1] for k in search_parameters)

3467

In [16]:
remaining = [s for s in sets if ('N0' not in s) and ('N1' not in s) 
             and ('N12306' not in s) and ('N1045' not in s)
             and ('N8194' not in s) and ('N17' not in s)
             and ('N9333' not in s) and ('N8969' not in s)
             and ('N19484' not in s) and ('N8227' not in s)
             and ("N2061" not in s) and ('N8833' not in s)]

In [273]:
remaining = [s for s in sets if ('N0' not in s)]

In [274]:
flat_list = [item for sublist in remaining for item in sublist]

In [279]:
strategies = list(set(flat_list))

In [281]:
strategies = sorted(strategies)

In [282]:
explains = []

for strategy in strategies:

    sets_explains = [i for i, s in enumerate(remaining) if strategy in s]

    explains.append(sets_explains)

In [283]:
len(explains)

35766

In [284]:
df = pd.DataFrame(explains)

df.to_csv(f"sets_per_strategy.csv", index=False)

In [267]:
int_remaining = [[int(element.split("N")[-1]) for element in s ] for s in remaining]

In [269]:
len(deterministic_strategies)

65536

In [270]:
flat_list = [item for sublist in int_remaining for item in sublist]

In [271]:
len(set(flat_list))

35766

In [99]:
flat_list = [item for sublist in remaining for item in sublist]

counter = collections.Counter(flat_list)

In [102]:
of_sets = []

for value in set(counter.values()):
    
    strategies = [key for key, count in zip(counter.keys(), counter.values()) if count == value]
    
    of_sets.append(set(strategies))

In [106]:
sum([len(list(s)) for s in of_sets])

35766

In [107]:
strategies_flat = [item for sublist in of_sets for item in sublist]

len(set(strategies_flat))

35766

In [157]:
random.seed(0)
random.choice?

In [160]:
data = []

for i in tqdm.notebook.tqdm(range(500)):
 
    random.seed(i)
    chosen_strategies = [random.choice(list(s), ) for s in of_sets]

    explains = []

    for strategy in chosen_strategies:

        strategy_explains = [i for i, s in enumerate(remaining) if strategy in s]

        explains.append(strategy_explains)
        
    indices = [item for sublist in explains for item in sublist]
    
    if len(set(indices)) > 85:
        
        data.append((len(set(indices)), i, chosen_strategies))

In [163]:
np.argmax([d[0] for d in data])

2

In [205]:
[i for i, d in enumerate(data) if d[0] == 91]

[2, 23, 34, 70, 103]

In [221]:
the_left_out_indices = []

num_of_indices = []

for i, d in tqdm.notebook.tqdm(enumerate(data)):
    
    chosen_strategies = d[-1]
    
    explains = []

    for strategy in chosen_strategies:

        strategy_explains = [i for i, s in enumerate(remaining) if strategy in s]

        explains.append(strategy_explains)
        
    indices = [item for sublist in explains for item in sublist]
    
    the_left_out_indices.append(set([i for i in range(94)]).difference(set(indices)))
    
    df = pd.DataFrame(explains)

    df.to_csv(f"sets_{i}.csv", index=False)
    
    import pandas as pd

    df = pd.read_csv(f"sets_{i}.csv")
    
    strategies = list(np.unique(df.values))[:-1]

    to_check = list(range(0, 50))

    explain = []
    the_max_set = list(df.iloc[-1].dropna().values)


    while (len(the_max_set) == len(strategies)) == False:
        Is = []
        for i in strategies:
            if i in the_max_set:
                pass
            else:
                Is.append(i)


        potential = []
        indices = []

        for j in to_check:
            for i in Is:
                if i in df.iloc[j].values:
                    indices.append(j)
                    potential.append(df.iloc[j].dropna().values)

        lengths = [len(set(list(the_max_set) + list(p))) for p in potential]


        index = indices[np.argmax(lengths)]
        to_check.remove(index)

        explain.append(index)

        the_max_set = list(set(list(the_max_set) + list(df.iloc[index].dropna().values)))
        
    num_of_indices.append(len(explain))

In [223]:
np.argmin(num_of_indices)

35

In [261]:
num_of_indices[16]

9

In [224]:
the_left_out_indices[35]

{19, 26, 28, 40, 50, 52, 68, 80}

In [ ]:
19, 28, 52, 68

In [241]:
a = set.intersection(*[set(remaining[26]), set(remaining[80]), set(remaining[50])])

In [253]:
set.intersection(*[set(remaining[52]), set(remaining[68])])

set()

In [206]:
chosen_strategies = data[23][-1]

In [207]:
explains = []

for strategy in chosen_strategies:

    strategy_explains = [i for i, s in enumerate(remaining) if strategy in s]

    explains.append(strategy_explains)

In [208]:
indices = [item for sublist in explains for item in sublist]

In [209]:
len(set(indices))

91

In [210]:
df = pd.DataFrame(explains)

df.to_csv("sets.csv", index=False)

In [211]:
indices = [item for sublist in explains for item in sublist]

In [212]:
set([i for i in range(94)]).difference(set(indices))

{12, 19, 71}

In [213]:
counter = collections.Counter(list(remaining[12]) + list(remaining[19]) + list(remaining[28]))

In [214]:
set(counter.values())

{1, 2}

In [215]:
set(remaining[12]).intersection(set(remaining[19]))

set()

In [216]:
set(remaining[19]).intersection(set(remaining[28]))

set()

In [217]:
_ = set(remaining[12]).intersection(set(remaining[28]))

In [191]:
remaining[19]

array(['N1125', 'N1127', 'N1133', 'N1135', 'N1141', 'N1143', 'N1149',
       'N1151', 'N1381', 'N1383', 'N1389', 'N1391', 'N1397', 'N1399',
       'N1405', 'N1407', 'N1637', 'N1639', 'N1645', 'N1647', 'N1653',
       'N1655', 'N1661', 'N1663', 'N1893', 'N1895', 'N1901', 'N1903',
       'N1909', 'N1911', 'N1917', 'N1919', 'N3173', 'N3175', 'N3181',
       'N3183', 'N3189', 'N3191', 'N3197', 'N3199', 'N3429', 'N3431',
       'N3437', 'N3439', 'N3445', 'N3447', 'N3453', 'N3455', 'N3685',
       'N3687', 'N3693', 'N3695', 'N3701', 'N3703', 'N3709', 'N3711',
       'N3941', 'N3943', 'N3949', 'N3951', 'N3957', 'N3959', 'N3965',
       'N3967', 'N9333', 'N9589', 'N9845', 'N10101', 'N17509', 'N17511',
       'N17517', 'N17519', 'N17525', 'N17527', 'N17533', 'N17535',
       'N17765', 'N17767', 'N17773', 'N17775', 'N17781', 'N17783',
       'N17789', 'N17791', 'N18021', 'N18023', 'N18029', 'N18031',
       'N18037', 'N18039', 'N18045', 'N18047', 'N18277', 'N18279',
       'N18285', 'N18287', 'N

In [185]:
assert len(set(remaining[19])) == len(set(remaining[12]).difference(remaining[12]))

assert len(set(remaining[19])) == len(set(remaining[12]).difference(remaining[28]))

AssertionError: 

In [181]:
s_for_three_remaining = [key for key, count in zip(counter.keys(), counter.values()) if count == 2]

In [ ]:
in_ = ()

for s in s_for_three_remaining:
        

In [154]:
indices = [item for sublist in explains for item in sublist]
len(set(indices))

87

In [60]:
explaining_indices = [30, 13, 14, 7, 27, 28]

In [87]:
one_strategy = [list(s)[0] for s in of_sets]

In [61]:
values_to_check = [list(set(counter.values()))[i] for i in explaining_indices]

In [69]:
strategies_set = []

for index in explaining_indices:
    
    strategies = of_sets[index]
    
    strategies_set.append(set(strategies))

In [70]:
import random

In [71]:
random_strategies = [list(s)[0] for s in strategies_set]

In [72]:
len(random_strategies)

6

In [88]:
[s for s in remaining if (one_strategy[30] not in s) and (one_strategy[13] not in s)
                          and (one_strategy[14] not in s) and (one_strategy[7] not in s)
                          and (one_strategy[27] not in s) and (one_strategy[28] not in s)]

[array(['N1', 'N9', 'N65', 'N73', 'N129', 'N137', 'N193', 'N201', 'N1025',
        'N1033', 'N1089', 'N1097', 'N1153', 'N1161', 'N1217', 'N1225',
        'N2049', 'N2057', 'N2113', 'N2121', 'N2177', 'N2185', 'N2241',
        'N2249', 'N3073', 'N3081', 'N3137', 'N3145', 'N3201', 'N3209',
        'N3265', 'N3273', 'N4097', 'N4105', 'N4161', 'N4169', 'N4225',
        'N4233', 'N4289', 'N4297', 'N5121', 'N5129', 'N5185', 'N5193',
        'N5249', 'N5257', 'N5313', 'N5321', 'N6145', 'N6153', 'N6209',
        'N6217', 'N6273', 'N6281', 'N6337', 'N6345', 'N7169', 'N7177',
        'N7233', 'N7241', 'N7297', 'N7305', 'N7361', 'N7369', 'N8193',
        'N8201', 'N8321', 'N8329', 'N9217', 'N9225', 'N9345', 'N9353',
        'N10241', 'N10249', 'N10369', 'N10377', 'N11265', 'N11273',
        'N11393', 'N11401', 'N12289', 'N12297', 'N12417', 'N12425',
        'N13313', 'N13321', 'N13441', 'N13449', 'N14337', 'N14345',
        'N14465', 'N14473', 'N15361', 'N15369', 'N15489', 'N15497',
        'N1638

In [84]:
list(of_sets[0])

['N20664',
 'N3562',
 'N23743',
 'N1061',
 'N3172',
 'N4856',
 'N20582',
 'N50540',
 'N23864',
 'N21678',
 'N17848',
 'N7344',
 'N1132',
 'N23014',
 'N18272',
 'N19772',
 'N3510',
 'N5565',
 'N19510',
 'N2808',
 'N426',
 'N51747',
 'N22884',
 'N18664',
 'N20576',
 'N7358',
 'N18488',
 'N19316',
 'N18912',
 'N21694',
 'N6568',
 'N17573',
 'N17855',
 'N21094',
 'N3370',
 'N18914',
 'N17726',
 'N57120',
 'N23520',
 'N4196',
 'N7408',
 'N21224',
 'N18793',
 'N491',
 'N21472',
 'N1145',
 'N2926',
 'N488',
 'N20768',
 'N19360',
 'N6114',
 'N23734',
 'N5483',
 'N4192',
 'N33302',
 'N49508',
 'N17512',
 'N7140',
 'N2428',
 'N4334',
 'N19436',
 'N17829',
 'N4336',
 'N18552',
 'N18656',
 'N2530',
 'N18356',
 'N22716',
 'N7402',
 'N1207',
 'N18798',
 'N23840',
 'N17846',
 'N57123',
 'N5874',
 'N7396',
 'N5241',
 'N34757',
 'N2544',
 'N7650',
 'N21484',
 'N3692',
 'N3322',
 'N7586',
 'N6768',
 'N56368',
 'N20720',
 'N56881',
 'N52784',
 'N21684',
 'N2146',
 'N4194',
 'N1084',
 'N19516',
 'N5438',


In [80]:
len([item for sublist in of_sets for item in sublist])

35766

In [81]:
len(set([item for sublist in of_sets for item in sublist]))

35766

In [82]:
one_strategy = [list(s)[0] for s in of_sets]

In [83]:
explains = []

for strategy in one_strategy:
    
    strategy_explains = [i for i, s in enumerate(remaining) if strategy in s]
    
    explains.append(strategy_explains)

In [ ]:
c = list(itertools.combinations(range(51), 10))
unq = set(c)

In [46]:
for i, j, k, l, m in tqdm.tqdm_notebook(unq):
    
    sets_ = set(explains[i] + explains[j] + explains[k] + explains[l] + explains[m])

    if len(sets_.difference(set(flat_list))) == 0:
        print(i, j)

<ipython-input-46-2dfa81bb66b8>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, j, k, l, m in tqdm.tqdm_notebook(unq):


KeyboardInterrupt: 

In [49]:
51 ** 5

345025251

In [34]:
for i, j, k in list(itertools.combinations(labels, r=3)):
    index = [s for _, s in enumerate(sets) if (i not in list(s) and j not in list(s) and k not in list(s))]
    
    if len(index) == 0:
        idx_sets.append((i, j, k))

[30, 75, 90]

In [17]:
# keys = [key for key, count in zip(counter.keys(), counter.values()) if count == set(counter.values())][0]

IndexError: list index out of range

In [ ]:
keys

In [233]:
remaining = [s for s in sets if ('N0' not in s)]

flat_list = [item for sublist in remaining for item in sublist]

counter = collections.Counter(flat_list)

In [234]:
counter['N8833']

26

In [235]:
[key for key, count in zip(counter.keys(), counter.values()) if count == 26]

['N19',
 'N83',
 'N147',
 'N211',
 'N267',
 'N717',
 'N901',
 'N961',
 'N1029',
 'N1043',
 'N1093',
 'N1157',
 'N1171',
 'N1221',
 'N1285',
 'N1291',
 'N1349',
 'N1413',
 'N1477',
 'N1541',
 'N1605',
 'N1669',
 'N1733',
 'N1797',
 'N1861',
 'N1925',
 'N1985',
 'N1989',
 'N2067',
 'N2195',
 'N2315',
 'N2637',
 'N2757',
 'N3077',
 'N3085',
 'N3091',
 'N3141',
 'N3149',
 'N3205',
 'N3213',
 'N3219',
 'N3269',
 'N3277',
 'N3339',
 'N4099',
 'N4741',
 'N4749',
 'N5123',
 'N5331',
 'N6147',
 'N6725',
 'N7171',
 'N7181',
 'N7245',
 'N7251',
 'N7309',
 'N7373',
 'N7379',
 'N8195',
 'N8219',
 'N8529',
 'N9219',
 'N9243',
 'N10243',
 'N10267',
 'N10705',
 'N11267',
 'N11291',
 'N12315',
 'N13339',
 'N13769',
 'N14363',
 'N15387',
 'N15809',
 'N16395',
 'N16403',
 'N16408',
 'N16467',
 'N16472',
 'N16536',
 'N16600',
 'N17161',
 'N17293',
 'N17413',
 'N17427',
 'N17477',
 'N17541',
 'N17605',
 'N17669',
 'N17733',
 'N17797',
 'N17811',
 'N17861',
 'N17925',
 'N17989',
 'N18053',
 'N18117',
 'N181

In [236]:
of_sets = []

for value in set(counter.values()):
    
    strategies = [key for key, count in zip(counter.keys(), counter.values()) if count == value]
    
    of_sets.append(set(strategies))

In [237]:
pairs = []

for i, set_ in enumerate(of_sets[:-1]):
    for j, other_set in enumerate(of_sets[i + 1:]):
        if len(set_) == len(set_.difference(other_set)):
            pass
        else:
            pairs.append((i, j))

In [240]:
of_sets[-1]

{'N1',
 'N1025',
 'N10369',
 'N1089',
 'N11393',
 'N1153',
 'N1217',
 'N129',
 'N16385',
 'N16449',
 'N16513',
 'N16577',
 'N17409',
 'N17473',
 'N17537',
 'N17601',
 'N18433',
 'N18497',
 'N18561',
 'N18625',
 'N193',
 'N19457',
 'N19521',
 'N19585',
 'N19649',
 'N2049',
 'N2113',
 'N2177',
 'N2241',
 'N3073',
 'N3137',
 'N3201',
 'N3265',
 'N65',
 'N8321',
 'N9345'}

In [204]:
len(of_sets[-1].difference(of_sets[-2]))

36

In [205]:
len(of_sets[-1])

36

In [113]:
# remaining = [[int(item.split("N")[-1]) for item in s] for s in remaining]

In [114]:
len(remaining)

47

In [123]:
lenghts = [len(s) for s in remaining]

In [126]:
sorted(lenghts)

[72,
 128,
 147,
 180,
 217,
 222,
 224,
 230,
 288,
 300,
 336,
 336,
 374,
 406,
 440,
 443,
 525,
 528,
 598,
 632,
 680,
 805,
 809,
 867,
 932,
 934,
 971,
 996,
 1001,
 1031,
 1058,
 1081,
 1106,
 1125,
 1169,
 1207,
 1233,
 1253,
 1358,
 1359,
 1389,
 1481,
 1482,
 1483,
 1506,
 1553,
 1627,
 1681,
 1720,
 1784,
 1930,
 1947,
 1956,
 1987,
 2253,
 2275,
 2398,
 2418,
 2467,
 2493,
 2881,
 2995,
 3111,
 3310,
 3396,
 3424,
 3497,
 3524,
 3974,
 4206,
 4400,
 5578,
 5941,
 6098,
 6423,
 7060,
 7205,
 7250,
 8077,
 8139,
 8685,
 8700,
 9130,
 9208,
 9925,
 10843,
 12472,
 12498,
 12738,
 14026,
 18061,
 19043,
 21329,
 30089]

In [116]:
np.argmin(lenghts)

32

In [118]:
[counter[s] for s in remaining[32]]

[11,
 11,
 11,
 11,
 8,
 8,
 8,
 8,
 11,
 11,
 11,
 11,
 8,
 8,
 8,
 8,
 11,
 11,
 11,
 11,
 8,
 8,
 8,
 8,
 11,
 11,
 11,
 11,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 9,
 9,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 8,
 8,
 8,
 8,
 11,
 11,
 11,
 11,
 8,
 8,
 8,
 8,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 9,
 8,
 9,
 9,
 8,
 8,
 9,
 6,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 9,
 9,
 9,
 9,
 9,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 9,
 9,
 9]

In [108]:
remaining[32][3]

'N8206'

In [109]:
counter['N592']

4

In [ ]:
flat_list = [item for sublist in remaining for item in sublist]

In [ ]:
max(collections.Counter(flat_list).values())

In [ ]:
values = set(collections.Counter(flat_list).values())

In [ ]:
idx_sets = []

combis = []

lenghts = []

for i, j in tqdm.tqdm_notebook(itertools.combinations(values, r=2)):
    
    sum_ = i + j
    
    if sum_ >= 94 :
        
        key_one = [key for key, value in zip(counts.keys(), counts.values()) if value == i][0]
        key_two = [key for key, value in zip(counts.keys(), counts.values()) if value == j][0]
        
        check = [s for s in remaining if (key_one not in s)
         and (key_two not in s)]
        
        lenghts.append(len(check))


        
#         combis.append((i, j))



In [ ]:
min(lenghts)

In [ ]:
idx_sets = []

combis = []

for indices  in tqdm.tqdm_notebook(itertools.combinations(values, r=4)):
    
    sum_ = sum(indices)
    
    if sum_ >= 94 :
        
        combis.append((indices))


In [ ]:
len(combis)

In [ ]:
counts = collections.Counter(flat_list)

In [ ]:
for indices in combis:
    keys = []
    
    for index in indices:
        keys.append([key for key, value in zip(counts.keys(), counts.values()) if value == index][0])
        
#     check = [s for s in remaining if (key not in s) for key in keys]



#                     if len(check) == 0:
#                         print(key_one, key_two, key_three, key_four)

In [ ]:
for i, j, k, z in tqdm.tqdm_notebook(combis[:1]):
    for key_one in [key for key, value in zip(counts.keys(), counts.values()) if value == i][:1]:
        for key_two in [key for key, value in zip(counts.keys(), counts.values()) if value == j][:1]:
            for key_three in [key for key, value in zip(counts.keys(), counts.values()) if value == k][:1]:
                for key_four in [key for key, value in zip(counts.keys(), counts.values()) if value == z][:1]:


                    check = [s for s in remaining if (key_one not in s)
                             and (key_two not in s) and 
                             (key_three not in s)  and 
                             (key_four not in s)]

                    if len(check) == 0:
                        print(key_one, key_two, key_three, key_four)

In [ ]:
next_remaining = [s for s in remaining if (1 not in s)
                  and (12306 not in s) and (1045 not in s)
                  and (8194 not in s) and (17 not in s)
                 and (9333 not in s) and (8969 not in s)
                 and (19484) not in s]

In [ ]:
next_remaining = [s for s in remaining if (18441 not in s)]

In [ ]:
len(next_remaining)

In [ ]:
collections.Counter(flat_list)[18441], 62771, 16315

In [ ]:
next_remaining[2]

In [ ]:
flat_list = [item for sublist in next_remaining for item in sublist]

In [ ]:
max(collections.Counter(flat_list).values())

In [ ]:
[key for key, value in zip(collections.Counter(flat_list).keys(), collections.Counter(flat_list).values()) 
 if value == 2][:1]

In [ ]:
for i, j, k in tqdm.tqdm_notebook(combis[:1]):
    for key_one in [key for key, value in zip(counts.keys(), counts.values()) if value == i]:
        for key_two in [key for key, value in zip(counts.keys(), counts.values()) if value == j]:
            for key_three in [key for key, value in zip(counts.keys(), counts.values()) if value == k]:
                check = [s for s in remaining if (key_one not in s) and (key_two not in s) and (key_three not in s)]

                if len(check) == 0:
                    print(key_one, key_two, key_three)

In [ ]:
[key for key, value in zip(counts.keys(), counts.values()) if value == 43]

In [ ]:
[key for key, value in zip(counts.keys(), counts.values()) if value == 51]

In [ ]:
len(remaining) - 51 - 12

In [ ]:
counts = collections.Counter(flat_list)

In [ ]:
list_counts = []
how_many = []
for i in [key for key, value in zip(counts.keys(), counts.values()) if value == 51]:
    next_remaining = [s for s in remaining if i not in s]
    
    flat_list = [item for sublist in next_remaining for item in sublist]
    
    list_counts.append(collections.Counter(flat_list))

In [ ]:
[key for key, value in zip(list_counts[0].keys(), list_counts[0].values()) if value == 12]

In [ ]:
len([s for s in remaining if 1 not in s])

In [ ]:
max([max(s) for s in remaining])

In [ ]:
len(remaining)

In [ ]:
np.argmax([len(s) for s in remaining])

In [ ]:
len(remaining[30])

In [ ]:
flat_list = [item for sublist in remaining for item in sublist]

In [ ]:
len(set(flat_list)) * 72

In [ ]:
labels = remaining[52]

idx_sets = []

for i in tqdm.tqdm_notebook(remaining[52]):
    for j in remaining[30]:

        index = [s for _, s in enumerate(remaining) if (i not in list(s) and j not in list(s))]

        if len(index) == 0:
            idx_sets.append((i, j))

In [ ]:
len(list(itertools.combinations(labels, r=2)))

In [ ]:
len(deterministic_strategies)

In [ ]:
len(remaining)

In [ ]:
flat_list = [item for sublist in remaining for item in sublist]

In [ ]:
import collections

In [ ]:
193: 51, 1: 51

In [ ]:
collections.Counter(flat_list)

In [ ]:
remaining = [[int(item.split("N")[-1]) for item in s] for s in remaining]

In [ ]:
set([s[0] for s in remaining])

In [ ]:
after_19461 = [[item for item in s if item >= 19461] for s in remaining]

In [ ]:
len(after_19461)

In [ ]:
[index for index, s in enumerate(after_19461) if len(s) == 0]

In [ ]:
remaining[69]

In [ ]:
set([s[0] for s in after_19461])

In [ ]:
[s for s in sets if ('N0' not in s) and ('N19461' not in s)]

In [ ]:
for number in tqdm.tqdm_notebook(range(len(deterministic_strategies))):
    set_ = [s for s in sets if ('N0' not in s) and (f'N{number}' not in s)]
    
    if len(set_) == 0:
        print(number)

In [ ]:
[s for s in sets if ('N0' not in s) and ('N' not in s)]

**The curve/plane**

In [ ]:
deterministic_strategies[65329]

In [ ]:
player

In [ ]:
p_1, p_2, p_3, p_4 = sym.symbols("p_1, p_2, p_3, p_4")

p_1 = 1

player = [p_1, p_2, p_1, p_2, p_3, p_4, p_3, p_4,
          p_1, p_2, p_1, p_2, p_3, p_4, p_3, p_4]

b, c = sym.symbols("b, c")

In [ ]:
coplayer = [1] + [0 for _ in range(15)]

In [ ]:
M = calculate_M_memory_two(coplayer, player, analytical=True)

In [ ]:
sym.init_printing(False)

# M
M

In [ ]:
ss = invariant_distribution_analytically(M)

In [ ]:
ss

In [ ]:
size = M.shape[1]
pi = sym.symbols(f"v_1:{size + 1}")
ss = sym.solve(
    [sum(pi) - 1]
    + [a - b for a, b in zip(M.transpose() * sym.Matrix(pi), pi)],
    pi,
)

In [ ]:
[a - b for a, b in zip(M.transpose() * sym.Matrix(pi), pi)]

**Plotting**

In [ ]:
from mpl_toolkits import mplot3d

import numpy as np

import matplotlib.pyplot as plt

import matplotlib

plt.rcParams["font.size"] = 15
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from matplotlib.lines import Line2D

In [ ]:
p_1, p_2, p_3, p_4 = sym.symbols("p_1, p_2, p_3, p_4")

b, c = sym.symbols("b, c")

p_1 = 1

p_2 = p_2

p_4 = p_4

b_val = 2
c_val = 1

p_3 = p_3 # 1 + (b_val - c_val) / b_val - p_2

player = [p_1, p_2, p_1, p_2, p_3, p_4, p_3, p_4,
          p_1, p_2, p_1, p_2, p_3, p_4, p_3, p_4]



In [ ]:
M

In [ ]:
indices

In [ ]:
states_in_two_bits = list(itertools.product(['C', 'D'], repeat=4))

In [ ]:
[states_in_two_bits[i] for i in indices]

In [ ]:
M = sym.Matrix([[p_1, 0, 0], [1, 0, 0], [0.5, 0.5, 0]])

In [ ]:
player

In [ ]:
coplayer = [0, 1, 1, 0] * 4 #deterministic_strategies[65331]
# coplayer = [1, 1, 0, 0] * 4
coplayer

In [ ]:
M = calculate_M_memory_two(coplayer, player, analytical=True)

In [ ]:
ss = invariant_distribution_analytically(M)

In [ ]:
sym.init_printing(True)

(ss @ np.array([b -c, -c, b, 0] * 4))[0].factor()

In [ ]:
(ss @ np.array([b -c, -c, b, 0] * 4))[0].factor()

In [ ]:
# check if alternator + ALLD explains all nash in memory-one

In [ ]:
M = calculate_M([0, 0, 0, 0], [1, p_2, p_3, p_4])

In [ ]:
(invariant_distribution_analytically(M) @ np.array([b - c, -c, b, 0]))[0]

In [ ]:
pure_mem_one = list(itertools.product([0, 1], repeat=4))

In [ ]:
satisfy =[]

for coplayer in tqdm.tqdm_notebook(pure_mem_one):
    
    M = calculate_M(coplayer, [1, p_2, p_3, p_4])
    
    try:
    
        score = (invariant_distribution_analytically(M) @ np.array([b - c, -c, b, 0]))[0]

        solution = sym.solve(score.factor() - (b - c), p_2)

        if solution:

            if  solution[0] - (b - c*p_3)/b == 0:
                satisfy.append(coplayer)
                
    except KeyError:
        pass

In [ ]:
satisfy

In [ ]:
M = calculate_M([0, 1, 1, 0], [1, p_2, p_3, p_4])
score = (invariant_distribution_analytically(M) @ np.array([b - c, -c, b, 0]))[0]

In [ ]:
sym.solve(score - (b -c), p_2)

In [ ]:
M = calculate_M([0, 0, 0, 1], [1, p_2, p_3, p_4])
score = (invariant_distribution_analytically(M) @ np.array([b - c, -c, b, 0]))[0]

In [ ]:
[i.factor() for i in ss]

In [ ]:
[i for i, _ in enumerate(ss) if ]

In [ ]:
M_num = np.array(M)

In [ ]:
M_num[indices, indices]

In [ ]:
indices = [2, 3, 6, 7, 8, 9, 12, 13] 

In [ ]:
T = sym.zeros(len(indices), len(indices))

for i, index1 in enumerate(indices):
    for j, index2 in enumerate(indices):
            T[i, j] = M[index1, index2]

In [ ]:
T

In [ ]:
ss.subs({p_2: 0.9, p_3:0.23})

In [ ]:
ss.subs({p_2: 0.9, p_3:0.23}) @ T.subs({p_2: 0.9, p_3:0.23})

In [ ]:
ss_fake

In [ ]:
[i.factor() for i in ss_fake @ M]

In [ ]:
ss_fake @ M

In [ ]:
sym.solve(((ss_fake @ np.array([b - c, -c, b, 0] * 4))[0].factor() - (b - c)).factor(), p_2)

In [ ]:
(ss_fake @ np.array([b - c, -c, b, 0] * 4))[0].simplify()

In [ ]:
ss_fake = sym.zeros(1, 16)

In [ ]:
ss = invariant_distribution_analytically(T)

for i, index in enumerate(indices):
    ss_fake[index] = ss[i]

In [ ]:
sum(ss_fake).factor()

In [ ]:
M

In [ ]:
ss = invariant_distribution_analytically(M)

In [ ]:
M = calculate_M_memory_two(coplayer, player)

ss = invariant_distribution(M)

In [ ]:
ss @ np.array([b - c, -c, b, 0] * 4)

In [ ]:
stationaries = []

eigenvalues, eigenvectors = np.linalg.eig(M.T)

for index in np.where(np.isclose(eigenvalues, 1))[0]:

    eigenvectors_one = eigenvectors[:, index]

    stationary = eigenvectors_one / eigenvectors_one.sum()


    stationaries.append(stationary.real)

In [ ]:
[index for index, val in enumerate(stationaries[0]) if val != 0]

In [ ]:
M

In [ ]:
M = sym.Matrix([[p_1, 0, 0], [1, 0, 0], [0.5, 0.5, 0]])

In [ ]:
sym.Matrix(M)

In [ ]:
(ss @ sym.Matrix([b - c, -c , b, 0] * 4))

In [ ]:
x = np.arange(0, 1 - 10 ** -5, 0.02)
y = np.arange(0, 1 - 10 ** -5, 0.02)
z = np.arange(0, 1 - 10 ** -5, 0.02)

In [ ]:
deterministic_strategies[0]

In [ ]:
data = []

coplayer = deterministic_strategies[0]

for i in tqdm.tqdm_notebook(x):
    for j in y:
        for k in z:
            p = (1, i, 1, i, j, k, j, k, 1, i, 1, i, j, k, j, k)
            
            M = calculate_M_memory_two(coplayer, p)
            
            
            ss = invariant_distribution(M)


            score = (ss @ np.array([2 - 1, -1, 2, 0] * 4)) - (2 - 1)
            
            if np.isclose(score, 0, atol=10 ** - 2):
                data.append((i, j, k))

In [ ]:
x = np.arange(0, 1 - 10 ** -5, 0.02)
y = np.arange(0, 1 - 10 ** -5, 0.02)
z = np.arange(0, 1 - 10 ** -5, 0.02)

In [ ]:
data65331 = []

coplayer = deterministic_strategies[65331]

for i in tqdm.tqdm_notebook(x):
    for j in y:
        for k in z:
            p = (1, i, 1, i, j, k, j, k, 1, i, 1, i, j, k, j, k)
            
            M = calculate_M_memory_two(coplayer, p)
            
            
            ss = invariant_distribution(M)


            score = (ss @ np.array([5 - 4, -4, 5, 0] * 4)) - (5 - 4)
            
            if np.isclose(score, 0, atol=10 ** - 2):
                data65331.append((i, j, k))

In [ ]:
def N65330(p, b, c):
    _, p_2, p_3, p_4 = p
    
    return p_2 + p_3 - (1 + (b - c) / b)


def check(p, b, c):
    _, p_2, p_3, p_4 = p
    
    return (b - c * p_2) / b  - p_3

In [ ]:
x = np.arange(0, 1 - 10 ** -5, 0.05)
y = np.arange(0, 1 - 10 ** -5, 0.05)
z = np.arange(0, 1 - 10 ** -5, 0.05)

xv, yv, zv = np.meshgrid(x, y, z)

In [ ]:
Is65330, Js65330, Zs65330 = np.where(np.isclose(N65330((1, xv, yv, zv), 2, 1), 0, atol=10 ** - 3))

Ischeck, Jscheck, Zscheck = np.where(np.isclose(check((1, xv, yv, zv), 2, 1), 0, atol=10 ** - 3))

In [ ]:
subset = eqmemtwo[(eqmemtwo['p2'] > .5) & (eqmemtwo['p3'] > .5)]

In [ ]:
%matplotlib notebook

# %matplotlib inline


# custom_lines = [Line2D([0], [0], color='tab:purple', lw=4, alpha=.55)]

# figsize=(20, 20)

fig = plt.figure(figsize=(15, 15))

ax1 = fig.add_subplot(131, projection='3d')

axes = [ax1]

for i, row in subset[['p2', 'p5', 'p6']].round(2).drop_duplicates().iterrows():

        ax1.scatter(row['p2'], row['p5'], row['p6'], color='tab:blue', alpha=0.5)

ax1.set_zlim(0, 1);

        
# for i, j, z in zip(Is65330, Js65330, Zs65330):

#     ax1.scatter(xv[i, j, z],
#                     yv[i, j, z],
#                     zv[i, j, z],
#                     s=1,
#                     color='purple')

# ax1.scatter(xv[i, j, z],
#         yv[i, j, z],
#         zv[i, j, z],
#         s=1,
#         color='purple')

for i, j, z in zip(Ischeck, Jscheck, Zscheck):

    ax1.scatter(xv[i, j, z],
                    yv[i, j, z],
                    zv[i, j, z],
                    s=1,
                    color='green')

ax1.scatter(xv[i, j, z],
        yv[i, j, z],
        zv[i, j, z],
        s=1,
        color='green')
        
        
for ax in axes:

#     ax.view_init(10, -70)
    ax.view_init(10, -60)
    ax.set_xticks([0, .5, 1])
    ax.set_yticks([0, .5, 1])
    ax.set_zticks([0, .5, 1])

    ax.set_xticklabels([0, "", 1])
    ax.set_yticklabels([0, "", 1])
    ax.set_zticklabels([0, "", 1])


    ax.annotate("$p_2$", (-0.05, -0.08), fontsize=15)
    ax.annotate("$p_3$", (.065, -0.06), fontsize=15)
    ax.annotate("$p_4$", (.09, 0.003), fontsize=15)
    
    
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

# for point in data65331:
#      ax1.scatter(point[0],
#                  point[1],
#                  point[2],
#                  s=1,
#                  color='tab:purple')
        
    
# ax1.legend(custom_lines, ['N65331'], bbox_to_anchor=(.5, 0), ncol=4, framealpha=0, fontsize=12);

In [ ]:
min([point[0] for point in data65331]), max([point[0] for point in data65331]) 

In [ ]:
 custom_lines = [Line2D([0], [0], color='tab:purple', lw=4, alpha=.55)]

fig = plt.figure(figsize=(20, 20))

ax1 = fig.add_subplot(131, projection='3d')

axes = [ax1]


for i, row in eqmemtwo[['p2', 'p5', 'p6']].round(2).drop_duplicates().iterrows():

        ax1.scatter(row['p2'], row['p5'], row['p6'], color='tab:blue', alpha=0.5)

ax1.set_zlim(0, 1);


xx, yy= np.meshgrid(np.arange(0, 1, 0.1), np.arange(0, 1, 0.1))

z = 1  - xx

# plot the plane
ax1.plot_surface(xx, yy, z, alpha=0.5)
        
for ax in axes:

#     ax.view_init(10, -70)
    ax.view_init(10, -80)
    ax.set_xticks([0, .5, 1])
    ax.set_yticks([0, .5, 1])
    ax.set_zticks([0, .5, 1])

    ax.set_xticklabels([0, "", 1])
    ax.set_yticklabels([0, "", 1])
    ax.set_zticklabels([0, "", 1])


    ax.annotate("$p_2$", (-0.05, -0.08), fontsize=15)
    ax.annotate("$p_3$", (.065, -0.06), fontsize=15)
    ax.annotate("$p_4$", (.09, 0.003), fontsize=15)
    
    
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

In [ ]:
%matplotlib inline


fig = plt.figure(figsize=(20, 20))

ax1 = fig.add_subplot(131, projection='3d')

axes = [ax1]


# for point in data:
#      ax1.scatter(point[0],
#                  point[1],
#                  point[2],
#                  s=1,
#                  color='tab:purple')

# do fit
tmp_A = []
tmp_b = []

for point in data:
    
    tmp_A.append([point[0], point[1], 1])
    tmp_b.append(point[2])
    
b = np.matrix(tmp_b).T

A = np.matrix(tmp_A)

fit = (A.T * A).I * A.T * b

errors = b - A * fit

residual = np.linalg.norm(errors)

print("solution:")
print("%f x + %f y + %f = z" % (fit[0], fit[1], fit[2]))

xlim = ax1.get_xlim()
ylim = ax1.get_ylim()

X, Y = np.meshgrid(np.arange(xlim[0], xlim[1], 0.1),
                   np.arange(ylim[0], ylim[1], 0.1))

Z = np.zeros(X.shape)

for r in range(X.shape[0]):
    for c in range(X.shape[1]):
        Z[r, c] = fit[0] * X[r, c] + fit[1] * Y[r, c] + fit[2]

ax1.plot_wireframe(X, Y, Z, color='k')

for ax in axes:

#     ax.view_init(10, -70)
    ax.view_init(10, -60)
    ax.set_xticks([0, .5, 1])
    ax.set_yticks([0, .5, 1])
    ax.set_zticks([0, .5, 1])

    ax.set_xticklabels([0, "", 1])
    ax.set_yticklabels([0, "", 1])
    ax.set_zticklabels([0, "", 1])


    ax.annotate("$p_2$", (-0.05, -0.08), fontsize=15)
    ax.annotate("$p_3$", (.065, -0.06), fontsize=15)
    ax.annotate("$p_4$", (.09, 0.003), fontsize=15)
    
    
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False;

In [ ]:
%matplotlib inline


fig = plt.figure(figsize=(20, 20))

ax1 = fig.add_subplot(131, projection='3d')

axes = [ax1]


for point in data65331:
     ax1.scatter(point[0],
                 point[1],
                 point[2],
                 s=1,
                 color='tab:purple')
        

# for ax in axes:

# #     ax.view_init(10, -70)
#     ax.view_init(10, -60)
#     ax.set_xticks([0, .5, 1])
#     ax.set_yticks([0, .5, 1])
#     ax.set_zticks([0, .5, 1])

#     ax.set_xticklabels([0, "", 1])
#     ax.set_yticklabels([0, "", 1])
#     ax.set_zticklabels([0, "", 1])


#     ax.annotate("$p_2$", (-0.05, -0.08), fontsize=15)
#     ax.annotate("$p_3$", (.065, -0.06), fontsize=15)
#     ax.annotate("$p_4$", (.09, 0.003), fontsize=15)
    
    
#     ax.xaxis.pane.fill = False
#     ax.yaxis.pane.fill = False
#     ax.zaxis.pane.fill = False;

# do fit
tmp_A = []
tmp_b = []

for point in data65331:
    
    tmp_A.append([point[1], point[2], 1])
    tmp_b.append(point[0])

b = np.matrix(tmp_b).T

A = np.matrix(tmp_A)


fit = (A.T * A).I * A.T * b

errors = b - A * fit

residual = np.linalg.norm(errors)

print("solution:")
print("%f p3 + %f p4 + %f = p2" % (fit[0], fit[1], fit[2]))

xlim = ax1.get_xlim()
ylim = ax1.get_ylim()

X, Y = np.meshgrid(np.arange(xlim[0], xlim[1], 0.1),
                   np.arange(ylim[0], ylim[1], 0.1))

Z = np.zeros(X.shape)

for r in range(X.shape[0]):
    for c in range(X.shape[1]):
        Z[r, c] = fit[0] * X[r, c] + fit[1] * Y[r, c] + fit[2]

ax1.plot_wireframe(Z, X, Y, color='k');


In [ ]:
player

In [ ]:
b, c = sym.symbols("b, c")

In [ ]:
sym.init_printing(True)

In [ ]:
possible = []

for coplayer in tqdm.tqdm_notebook(deterministic_strategies):
    
    M = calculate_M_memory_two(coplayer, player, analytical=True)
    
    try:
        ss = invariant_distribution_analytically(M)
    except KeyError:
        ss = False
        
    if ss:
        score = ss.subs({p_3:1}) @ np.array([b - c, -c, b, 0] * 4)
        
        if (score[0] - b) == 0:
            possible.append(coplayer)

In [ ]:
possible

In [ ]:
coplayer = [1, 1, 0, 0] * 4

coplayer

In [ ]:
M = calculate_M_memory_two(coplayer, player, analytical=True)

In [ ]:
ss = invariant_distribution_analytically(M)

In [ ]:
(ss.subs({p_4:1}) @ np.array([b - c, -c, b, 0] * 4))[0]

In [ ]:
player = [1, p_2, 1, p_2, 1, p_4, 1, p_4, 1, p_2, 1, p_2, 1, p_4, 1, p_4]

M2 = calculate_M_memory_two(deterministic_strategies[65329], player, analytical=True)

ss = invariant_distribution_analytically(M2)

In [ ]:
M2

In [ ]:
# run simulations with prisoner's dilemma
# for mem two see if there is a small subset
